In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci

def get_triangle_coords(d):
    """Returns coordinates for H3+ in equilateral triangle arrangement"""
    h = d * np.sqrt(3) / 2
    return [[0,0,0], [d,0,0], [d/2,h,0]]

def get_pyscf_mol(coords, charge, spin, basis):
    atom_spec = ""
    for i, c in enumerate(coords):
        atom_spec += f"H {c[0]} {c[1]} {c[2]}; "
    atom_spec = atom_spec.strip("; ")
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol

def get_qiskit_problem(coords, charge, spin, basis):
    atom_spec = ""
    for c in coords:
        atom_spec += f"H {c[0]} {c[1]} {c[2]}; "
    atom_spec = atom_spec.strip("; ")
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM, charge=charge, spin=spin, basis=basis)
    return driver.run()

def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf

def run_fci_pyscf(mol):
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci

def run_vqe_cpu(problem, mapper):
    """VQE function using CPU"""
    # Use standard Qiskit Estimator (CPU-based)
    estimator = Estimator()
    
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    
    # Better initial point (small random values)
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy

# --- Main Calculation Setup ---
basis = 'sto-3g'
charge = 1
spin = 0
dists = np.linspace(0.4, 4.0, 50)  # Focus on relevant H3+ distances

vqe_energies, uhf_energies, fci_energies = [], [], []

# Loop through different H-H distances
for i, d in enumerate(dists):
    print(f"Calculating H3+ at d = {d:.2f} Å ({i+1}/{len(dists)})")
    coords = get_triangle_coords(d)
    
    # Classical reference calculations
    mol = get_pyscf_mol(coords, charge, spin, basis)
    uhf_energy = run_uhf_pyscf(mol)
    fci_energy = run_fci_pyscf(mol)
    
    # VQE calculation
    problem = get_qiskit_problem(coords, charge, spin, basis)
    mapper = ParityMapper(num_particles=problem.num_particles)
    vqe_energy = run_vqe_cpu(problem, mapper)
    
    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"  VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")

# Find minimum energy and corresponding distance
min_idx = np.argmin(vqe_energies)
min_energy = vqe_energies[min_idx]
eq_distance = dists[min_idx]

# Plot potential energy curves
# Plot potential energy curves with minimalist styling
plt.figure(figsize=(9, 5))

# Plot curves with thin lines and no markers
plt.plot(dists, vqe_energies, '-', label='VQE', color='green', linewidth=1)
plt.plot(dists, uhf_energies, '-', label='UHF', color='red', linewidth=1)
plt.plot(dists, fci_energies, '--', label='FCI', color='black', linewidth=1)

plt.xlabel("H-H Distance (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title("H$_3^+$ Potential Energy Surface (Equilateral Triangle)", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci


def get_h2_coords(d):
    """Returns coordinates for H2 molecule along z-axis"""
    return [[0, 0, -d / 2], [0, 0, d / 2]]


def get_pyscf_mol(coords, charge, spin, basis):
    atom_spec = ""
    for c in coords:
        atom_spec += f"H {c[0]} {c[1]} {c[2]}; "
    atom_spec = atom_spec.strip("; ")
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol


def get_qiskit_problem(coords, charge, spin, basis):
    atom_spec = ""
    for c in coords:
        atom_spec += f"H {c[0]} {c[1]} {c[2]}; "
    atom_spec = atom_spec.strip("; ")
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM,
                         charge=charge, spin=spin, basis=basis)
    return driver.run()


def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf


def run_fci_pyscf(mol):
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci


def run_vqe(problem, mapper):
    estimator = Estimator()
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0] * ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return problem.interpret(result).total_energies[0].real


# === Main Setup ===
basis = 'sto3g'
charge = 0
spin = 0
dists = np.linspace(0.4, 4.0, 50)

vqe_energies, uhf_energies, fci_energies = [], [], []

for d in dists:
    coords = get_h2_coords(d)

    # VQE
    try:
        problem = get_qiskit_problem(coords, charge, spin, basis)
        mapper = ParityMapper(num_particles=problem.num_particles)
        vqe_energy = run_vqe(problem, mapper)
    except Exception as e:
        vqe_energy = np.nan
        print(f"VQE failed at d={d:.2f}: {e}")

    # UHF
    try:
        mol = get_pyscf_mol(coords, charge, spin, basis)
        uhf_energy = run_uhf_pyscf(mol)
    except Exception as e:
        uhf_energy = np.nan
        print(f"UHF failed at d={d:.2f}: {e}")

    # FCI
    try:
        fci_energy = run_fci_pyscf(mol)
    except Exception as e:
        fci_energy = np.nan
        print(f"FCI failed at d={d:.2f}: {e}")

    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"d={d:.2f} | VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")


# === Plotting ===
plt.figure(figsize=(6, 4.5))
plt.plot(dists, vqe_energies, label='VQE', color='limegreen')
plt.plot(dists, uhf_energies, label='UHF', color='red')
plt.plot(dists, fci_energies, '--', label='FCI', color='black')
plt.xlabel("H-H Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title("H$_2$ Potential Energy Curve")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci


def get_lih_coords(d):
    """Returns Li-H linear geometry at a given bond distance d (Å)"""
    return [["H", [0.0, 0.0, 0.0]], ["Li", [0.0, 0.0, d]]]


def get_pyscf_mol(coords, charge, spin, basis):
    mol = gto.Mole()
    mol.atom = coords
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol


def get_qiskit_problem(coords, charge, spin, basis):
    atom_str = "; ".join([f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}" for atom in coords])
    driver = PySCFDriver(atom=atom_str, unit=DistanceUnit.ANGSTROM,
                         charge=charge, spin=spin, basis=basis)
    return driver.run()


def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf


def run_fci_pyscf(mol):
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci


def run_vqe(problem, mapper):
    estimator = Estimator()
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0] * ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return problem.interpret(result).total_energies[0].real


# === Main Setup ===
basis = 'sto3g'
charge = 0
spin = 0
dists = np.linspace(0.5, 4.0, 50)

vqe_energies, uhf_energies, fci_energies = [], [], []

for d in dists:
    coords = get_lih_coords(d)

    # VQE
    try:
        problem = get_qiskit_problem(coords, charge, spin, basis)
        mapper = ParityMapper(num_particles=problem.num_particles)
        vqe_energy = run_vqe(problem, mapper)
    except Exception as e:
        vqe_energy = np.nan
        print(f"VQE failed at d={d:.2f}: {e}")

    # UHF
    try:
        mol = get_pyscf_mol(coords, charge, spin, basis)
        uhf_energy = run_uhf_pyscf(mol)
    except Exception as e:
        uhf_energy = np.nan
        print(f"UHF failed at d={d:.2f}: {e}")

    # FCI
    try:
        fci_energy = run_fci_pyscf(mol)
    except Exception as e:
        fci_energy = np.nan
        print(f"FCI failed at d={d:.2f}: {e}")

    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"d={d:.2f} | VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")


# === Plotting ===
plt.figure(figsize=(6, 4.5))
plt.plot(dists, vqe_energies, label='VQE', color='limegreen')
plt.plot(dists, uhf_energies, label='UHF', color='red')
plt.plot(dists, fci_energies, '--', label='FCI', color='black')
plt.xlabel("Li-H Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title("LiH Potential Energy Curve")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci

def get_oh_coords(d):
    """Returns coordinates for OH- diatomic molecule with O at origin"""
    return [["O", [0.0, 0.0, 0.0]], 
            ["H", [0.0, 0.0, d]]]

def get_pyscf_mol(coords, charge, spin, basis):
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol

def get_qiskit_problem(coords, charge, spin, basis):
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM, charge=charge, spin=spin, basis=basis)
    return driver.run()

def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf

def run_fci_pyscf(mol):
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci

def run_vqe_cpu(problem, mapper):
    """VQE function using CPU"""
    # Use standard Qiskit Estimator (CPU-based)
    estimator = Estimator()
    
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    
    # Better initial point (small random values)
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy

# --- Main Calculation Setup ---
basis = 'sto-3g'
charge = -1  # OH- has a negative charge
spin = 0     # Closed shell system
dists = np.linspace(0.5, 4.0, 50)  # Focus on relevant OH- distances

vqe_energies, uhf_energies, fci_energies = [], [], []

# Loop through different O-H distances
for i, d in enumerate(dists):
    print(f"Calculating OH- at d = {d:.2f} Å ({i+1}/{len(dists)})")
    coords = get_oh_coords(d)
    
    # Classical reference calculations
    mol = get_pyscf_mol(coords, charge, spin, basis)
    uhf_energy = run_uhf_pyscf(mol)
    fci_energy = run_fci_pyscf(mol)
    
    # VQE calculation
    problem = get_qiskit_problem(coords, charge, spin, basis)
    mapper = ParityMapper(num_particles=problem.num_particles)
    vqe_energy = run_vqe_cpu(problem, mapper)
    
    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"  VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")

# Plot potential energy curves with minimalist styling
plt.figure(figsize=(9, 5))

# Plot curves with thin lines and no markers
plt.plot(dists, vqe_energies, '-', label='VQE', color='green', linewidth=1)
plt.plot(dists, uhf_energies, '-', label='UHF', color='red', linewidth=1)
plt.plot(dists, fci_energies, '--', label='FCI', color='black', linewidth=1)

plt.xlabel("O-H Distance (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title("OH$^-$ Potential Energy Surface", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find minimum energy and corresponding distance
min_idx = np.argmin(vqe_energies)
min_energy = vqe_energies[min_idx]
eq_distance = dists[min_idx]

print(f"OH- ground state energy (VQE): {min_energy:.6f} Hartree")
print(f"Equilibrium O-H distance: {eq_distance:.3f} Å")
print(f"Energy difference VQE-FCI: {(min_energy-fci_energies[min_idx])*1000:.2f} mHartree")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci

def get_hf_coords(d):
    """Returns coordinates for HF diatomic molecule with H at origin"""
    return [["H", [0.0, 0.0, 0.0]], 
            ["F", [0.0, 0.0, d]]]

def get_pyscf_mol(coords, charge, spin, basis):
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol

def get_qiskit_problem(coords, charge, spin, basis):
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM, charge=charge, spin=spin, basis=basis)
    return driver.run()

def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf

def run_fci_pyscf(mol):
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci

def run_vqe_cpu(problem, mapper):
    """VQE function using CPU"""
    # Use standard Qiskit Estimator (CPU-based)
    estimator = Estimator()
    
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    
    # Better initial point (small random values)
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy

# --- Main Calculation Setup ---
basis = 'sto-3g'
charge = 0  # HF is neutral
spin = 0    # Closed shell system
dists = np.linspace(0.5, 4.0, 50)  # Focus on relevant HF distances

vqe_energies, uhf_energies, fci_energies = [], [], []

# Loop through different H-F distances
for i, d in enumerate(dists):
    print(f"Calculating HF at d = {d:.2f} Å ({i+1}/{len(dists)})")
    coords = get_hf_coords(d)
    
    # Classical reference calculations
    mol = get_pyscf_mol(coords, charge, spin, basis)
    uhf_energy = run_uhf_pyscf(mol)
    fci_energy = run_fci_pyscf(mol)
    
    # VQE calculation
    problem = get_qiskit_problem(coords, charge, spin, basis)
    mapper = ParityMapper(num_particles=problem.num_particles)
    vqe_energy = run_vqe_cpu(problem, mapper)
    
    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"  VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")

# Plot potential energy curves with minimalist styling
plt.figure(figsize=(9, 5))

# Plot curves with thin lines and no markers
plt.plot(dists, vqe_energies, '-', label='VQE', color='green', linewidth=1)
plt.plot(dists, uhf_energies, '-', label='UHF', color='red', linewidth=1)
plt.plot(dists, fci_energies, '--', label='FCI', color='black', linewidth=1)

plt.xlabel("H-F Distance (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title("HF Potential Energy Surface", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find minimum energy and corresponding distance
min_idx = np.argmin(vqe_energies)
min_energy = vqe_energies[min_idx]
eq_distance = dists[min_idx]

print(f"HF ground state energy (VQE): {min_energy:.6f} Hartree")
print(f"Equilibrium H-F distance: {eq_distance:.3f} Å")
print(f"Energy difference VQE-FCI: {(min_energy-fci_energies[min_idx])*1000:.2f} mHartree")